<a href="https://githubtocolab.com/rramosp/ai4eng_example_project/blob/main/03%20-%20Modelos-e-iteraciones.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open in Colab"/></a>

##1. Carga de dataset desde Kaggle

El dataset es tomado de la competencia de Kaggle: https://www.kaggle.com/competitions/allstate-claims-severity/data

In [ ]:
!pip install kaggle

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
#carga del token de kaggle

from google.colab import files 
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"tatianasanchez1","key":"89e2f6fbfe92a172129c5ee2b3bed732"}'}

In [ ]:
! mkdir ~/.kaggle

In [ ]:
! cp kaggle.json ~/.kaggle/

In [ ]:
! chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle competitions download -c allstate-claims-severity

 84% 41.0M/48.8M [00:00<00:00, 51.3MB/s]
100% 48.8M/48.8M [00:00<00:00, 53.1MB/s]


In [ ]:
!unzip allstate-claims-severity.zip


Archive:  allstate-claims-severity.zip
  inflating: sample_submission.csv   
  inflating: sample_submission.csv.zip  
  inflating: test.csv                
  inflating: test.csv.zip            
  inflating: train.csv               
  inflating: train.csv.zip           


## 2. Lectura y concatenación de los Dataset

In [ ]:
#cargar los datos
# Crear dataset 

train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')

In [ ]:
train.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14,loss
0,1,A,B,A,B,A,A,A,A,B,...,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843,2213.18
1,2,A,B,A,A,A,A,A,A,B,...,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496,1283.60
2,5,A,B,A,A,B,A,A,A,B,...,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425,3005.09
3,10,B,B,A,B,A,A,A,A,B,...,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642,939.85
4,11,A,B,A,B,A,A,A,A,B,...,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606,2763.85


In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 188318 entries, 0 to 188317
Columns: 132 entries, id to loss
dtypes: float64(15), int64(1), object(116)
memory usage: 189.7+ MB


Se observa que la base de datos de entrenamiento consta de 188,318 observaciones y 131 variables que incluyen 72 variables categóricas binarias, 43 variables no binarias, 14 variables continuas y la variable de resultado, "Loss"

In [ ]:
test.head()

,id,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,4,A,B,A,A,A,A,A,A,B,...,0.281143,0.466591,0.317681,0.61229,0.34365,0.38016,0.377724,0.369858,0.704052,0.392562
1,6,A,B,A,B,A,A,A,A,B,...,0.836443,0.482425,0.443760,0.71330,0.51890,0.60401,0.689039,0.675759,0.453468,0.208045
2,9,A,B,A,B,B,A,B,A,B,...,0.718531,0.212308,0.325779,0.29758,0.34365,0.30529,0.245410,0.241676,0.258586,0.297232
3,12,A,A,A,A,B,A,A,A,A,...,0.397069,0.369930,0.342355,0.40028,0.33237,0.31480,0.348867,0.341872,0.592264,0.555955
4,15,B,A,A,A,A,B,A,A,A,...,0.302678,0.398862,0.391833,0.23688,0.43731,0.50556,0.359572,0.352251,0.301535,0.825823


In [ ]:
test.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125546 entries, 0 to 125545
Columns: 131 entries, id to cont14
dtypes: float64(14), int64(1), object(116)
memory usage: 125.5+ MB


In [ ]:
"""
Checking the columns are identical or not in Train and Test data
"""

train_features = list(train.columns.values)
train_features.pop() #pop out the loss column
test_features = list(test.columns.values)
print('Is train and test datasets are identical? \n' + str(train_features == test_features))

Is train and test datasets are identical? 
True


## 3. Carga de librerias

In [ ]:
import warnings
warnings.filterwarnings("ignore")
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold
from xgboost import XGBRegressor
from sklearn.model_selection import RandomizedSearchCV,GridSearchCV
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression,Ridge,Lasso,ElasticNet
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor,BaggingRegressor,ExtraTreesRegressor,AdaBoostRegressor,GradientBoostingRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import SVR
from scipy.stats import skew,boxcox

In [ ]:
r,c = train.shape

In [ ]:
y_ = np.log1p(train['loss'])
train.drop(['id','loss'], axis=1, inplace=True)
test.drop(['id'], axis=1, inplace=True)

Train_Test = pd.concat((train, test)).reset_index(drop=True)

Train_Test.head()

,cat1,cat2,cat3,cat4,cat5,cat6,cat7,cat8,cat9,cat10,...,cont5,cont6,cont7,cont8,cont9,cont10,cont11,cont12,cont13,cont14
0,A,B,A,B,A,A,A,A,B,A,...,0.310061,0.718367,0.335060,0.30260,0.67135,0.83510,0.569745,0.594646,0.822493,0.714843
1,A,B,A,A,A,A,A,A,B,B,...,0.885834,0.438917,0.436585,0.60087,0.35127,0.43919,0.338312,0.366307,0.611431,0.304496
2,A,B,A,A,B,A,A,A,B,B,...,0.397069,0.289648,0.315545,0.27320,0.26076,0.32446,0.381398,0.373424,0.195709,0.774425
3,B,B,A,B,A,A,A,A,B,A,...,0.422268,0.440945,0.391128,0.31796,0.32128,0.44467,0.327915,0.321570,0.605077,0.602642
4,A,B,A,B,A,A,A,A,B,B,...,0.704268,0.178193,0.247408,0.24564,0.22089,0.21230,0.204687,0.202213,0.246011,0.432606


#Encoder

In [ ]:
from sklearn import preprocessing
le = preprocessing.LabelEncoder()

In [ ]:
categorical_ = [feature for feature in Train_Test.columns if 'cat' in feature]
continuous_ = [feature for feature in Train_Test.columns if 'cont' in feature]

#cat_feature = [n for n in joined.columns if n.startswith('cat')]    
#cont_feature = [n for n in joined.columns if n.startswith('cont')]

In [ ]:
for feature in categorical_: Train_Test[feature] = le.fit_transform(Train_Test[feature])

In [ ]:
from scipy.stats import skew,boxcox

In [ ]:
skewed_box = Train_Test.loc[:,"cont1":"cont14"].apply(lambda x: skew(x))
skewed_box

cont1     0.513205
cont2    -0.311146
cont3    -0.007023
cont4     0.417559
cont5     0.679610
cont6     0.458413
cont7     0.825889
cont8     0.673237
cont9     1.067247
cont10    0.352116
cont11    0.281139
cont12    0.291997
cont13    0.376138
cont14    0.250673
dtype: float64

In [ ]:
Train_Test["cont1"], lam = boxcox(Train_Test["cont1"] + 1)
Train_Test["cont2"], lam = boxcox(Train_Test["cont2"] + 1)  
Train_Test["cont4"], lam = boxcox(Train_Test["cont4"] + 1)
Train_Test["cont5"], lam = boxcox(Train_Test["cont5"] + 1)
Train_Test["cont6"], lam = boxcox(Train_Test["cont6"] + 1)
Train_Test["cont7"], lam = boxcox(Train_Test["cont7"] + 1)
Train_Test["cont8"], lam = boxcox(Train_Test["cont8"] + 1)
Train_Test["cont9"], lam = boxcox(Train_Test["cont9"] + 1)
Train_Test["cont10"], lam = boxcox(Train_Test["cont10"] + 1)
Train_Test["cont11"], lam = boxcox(Train_Test["cont11"] + 1)
Train_Test["cont12"], lam = boxcox(Train_Test["cont12"] + 1)
Train_Test["cont13"], lam = boxcox(Train_Test["cont13"] + 1)

In [ ]:
train = Train_Test.iloc[:r, :]
test = Train_Test.iloc[r:, :]

In [ ]:
# split the data into test and train by maintaining same distribution of output varaible 'y_true' [stratify=y_true]
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(train, y_,test_size=0.20)

## 4. Engineering

In [ ]:
# mean centering and unit variance.
def calculate_zscore(df,continous_features):

    # copying only the continous features from the dataframe
    df = df[continous_features].copy()
    for col in continous_features:
        df[col] = ( df[col] - df[col].mean())/df[col].std()
    return df

train['cont1']

0         0.467364
1         0.262984
2         0.217442
3         0.257281
4         0.225246
            ...   
188313    0.273618
188314    0.364971
188315    0.352724
188316    0.327463
188317    0.537725
Name: cont1, Length: 188318, dtype: float64

In [ ]:
calculate_zscore(X_train,continous_features).describe()

NameError: ignored

##XGBoost

In [ ]:
xg = XGBRegressor()

In [ ]:
prams={
    'learning_rate':[0.01,0.03,0.05,0.1,0.15,0.2],
     'n_estimators':[100,200,500,1000,2000],
}

In [ ]:
xgb_grid = GridSearchCV(xg,prams,cv = 3,scoring="neg_mean_squared_error",n_jobs = -1,verbose=True)

In [ ]:
xgb_grid.fit(X_train, y_train)

In [ ]:
results = pd.DataFrame.from_dict(xgb_grid.cv_results_)

In [ ]:
results.head(2)

In [ ]:
print(xgb_grid.best_params_)

In [ ]:
final_xg = XGBRegressor(base_score=0.5, booster='gbtree',
                                    colsample_bylevel=1, colsample_bynode=1,
                                    colsample_bytree=1, gamma=0,
                                    importance_type='gain', learning_rate=0.1,
                                    max_delta_step=0, max_depth=3,
                                    min_child_weight=1, missing=None,
                                    n_estimators=2000, n_jobs=1, nthread=None,
                                    objective='reg:linear', random_state=0,
                                    reg_alpha=0, reg_lambda=1,
                                    scale_pos_weight=1, seed=None, silent=None,
                                    subsample=1, verbosity=1).fit(X_train, y_train)

In [ ]:
y_pred_xg = np.expm1(final_xg.predict(X_test))
y_test_xg = np.expm1(y_test)
print(mean_absolute_error(y_pred_xg, y_test_xg))

##Gradient Boosting Regressor

In [ ]:

GBR = GradientBoostingRegressor()
prams={
    'learning_rate':[0.01,0.03,0.05,0.1,0.15,0.2],
     'n_estimators':[100,200,500,1000,2000],
     'max_depth':[1,2,4],
     'subsample':[.5,.75,1],
     'random_state':[1]
}

In [ ]:
gbr_grid = GridSearchCV(GBR,prams,cv = 3,scoring="neg_mean_squared_error",n_jobs = -1,verbose=True)

In [ ]:
gbr_grid.fit(X_train, y_train)

In [ ]:
print(gbr_grid.best_params_)

In [ ]:
GBR_final = GradientBoostingRegressor(alpha=0.9, ccp_alpha=0.0,
                                                 criterion='friedman_mse',
                                                 init=None, learning_rate=0.05,
                                                 loss='ls', max_depth=4,
                                                 max_features=None,
                                                 max_leaf_nodes=None,
                                                 min_impurity_decrease=0.0,
                                                 min_impurity_split=None,
                                                 min_samples_leaf=1,
                                                 min_samples_split=2,
                                                 min_weight_fraction_leaf=0.0,
                                                 n_estimators=2000,
                                                 subsample=0.75, tol=0.0001,
                                                 validation_fraction=0.1,
                                                 verbose=0, warm_start=False)

In [ ]:
GBR_final.fit(X_train, y_train)

In [ ]:
y_pred_xbr = np.expm1(GBR_final.predict(X_test))
y_test_xbr = np.expm1(y_test)
print(mean_absolute_error(y_pred_xbr, y_test_xbr))

###Lightbm

In [ ]:
lgb_r = LGBMRegressor(max_depth=-1, random_state=314, n_estimators=5000)
params = { 
  'boosting_type': ['gbdt','dart','goss','rf'], 
  'metric': ['l2', 'l1'], 
  'min_child_weight': [1e-5, 1e-3, 1e-2, 1e-1, 1, 1e1, 1e2, 1e3, 1e4], 
  'learning_rate': [0.01,0.03,0.05,0.1,0.15,0.2], 
  'reg_alpha': [1e-2, 1e-1, 1, 1e1, 1e2], 
  'reg_lambda': [1e-2, 1e-1, 1, 1e1, 1e2]
}
lgbmr_rs = RandomizedSearchCV(lgb_r,params,cv = 3,scoring="neg_mean_squared_error",n_job s = -1,verbose=True)
lgbmr_rs.fit(X_train, y_train)

In [ ]:
print(lgbmr_rs.best_params_)

In [ ]:
Train_Data = pd.read_csv('train.csv') 
Test_Data = pd.read_csv('test.csv')
r,c = Train_Data.shape   # r -> rows and c --> columns # 188318 and 13 
Train_Data.drop(['id','loss'], axis=1, inplace=True True) 
Test_Data.drop(['id'], axis=1, inplace=True True)
Train_Test = pd.concat((Train_Data, Test_Data)).reset_index(drop=True True)

categorical_ = [feature for for feature in in Train_Test.columns if if 'cat' in in feature] 
continuous_ = [feature for for feature in in Train_Test.columns if if 'cont' in in feature]

for feature in in categorical_: Train_Test[feature] = le.fit_transform(Train_Test[feature])

Train_Test["cont1"], lam = boxcox(Train_Test["cont1"] + 1) 
Train_Test["cont2"], lam = boxcox(Train_Test["cont2"] + 1)  
Train_Test["cont4"], lam = boxcox(Train_Test["cont4"] + 1) 
Train_Test["cont5"], lam = boxcox(Train_Test["cont5"] + 1) 
Train_Test["cont6"], lam = boxcox(Train_Test["cont6"] + 1) 
Train_Test["cont7"], lam = boxcox(Train_Test["cont7"] + 1) 
Train_Test["cont8"], lam = boxcox(Train_Test["cont8"] + 1) 
Train_Test["cont9"], lam = boxcox(Train_Test["cont9"] + 1) 
Train_Test["cont10"], lam = boxcox(Train_Test["cont10"] + 1) 
Train_Test["cont11"], lam = boxcox(Train_Test["cont11"] + 1) 
Train_Test["cont12"], lam = boxcox(Train_Test["cont12"] + 1) 
Train_Test["cont13"], lam = boxcox(Train_Test["cont13"] + 1)

Train_Data = Train_Test.iloc[:r, :] 
Test_Data = Train_Test.iloc[r:, :]

X_train, X_test, y_train, y_test = train_test_split(Train_Data, y_,test_size=0.20)

###Modelo final

In [ ]:
gridsearch_params = [
    (max_depth,subsample)
    for max_depth in range(4,18,4)
	for subsample in [n/10 for n in range(2,12,2)]
]
params = {}

"""
Reading Train and Test data
"""
Train_Data = pd.read_csv('train.csv')
Test_Data = pd.read_csv('test.csv')
Submission_Id = Test_Data['id']

r,c = Train_Data.shape   # r -> rows and c  --> columns # 188318 and 130

y = np.log(Train_Data['loss'] + 200)

Train_Data.drop(['id','loss'], axis=1, inplace=True)
Test_Data.drop(['id'], axis=1, inplace=True)

Train_Test = pd.concat((Train_Data, Test_Data)).reset_index(drop=True)

def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

if __name__ == '__main__':
  cat_feature = [n for n in Train_Data.columns if n.startswith('cat')]
  for column in cat_feature:
    if Train_Data[column].nunique() != Test_Data[column].nunique():
      Unique_classes_Train = set(Train_Data[column].unique())
      Unique_classes_Test = set(Test_Data[column].unique())
      missing_train = Unique_classes_Train.difference(Unique_classes_Test)           # set_A.difference(set_B) for (A - B)
      missing_test =  Unique_classes_Test.difference(Unique_classes_Train)


      All_misisng = missing_train.union(missing_test)

      # Replace all misisng categories with a common category instead of removing. 
      def missing_common(x):
        if x in All_misisng:
          return np.nan
        return x
      
      Train_Test[column] = Train_Test[column].apply(lambda x: missing_common(x), 1)   # Axis 1 :: columns
    
    Train_Test[column] = pd.factorize(Train_Test[column].values, sort=True)[0]

  Train_Data_final = Train_Test.iloc[:r, :]
  Test_Data_final = Train_Test.iloc[r:, :]  

  min_mae = float("Inf")

   best_params = None 

  xgtrain = xgb.DMatrix(Train_Data_final, label=y)

  for max_depth, subsample in gridsearch_params:
    print("CV with max_depth={}, subsample={}".format(max_depth, subsample))

    params['max_depth'] = max_depth
    params['subsample'] = subsample

    cv_results = xgb.cv(
        params,
        xgtrain,
        1000,
        seed=42,
        nfold=3,
        metrics={'mae'},
        early_stopping_rounds=10
    )


    mean_mae = cv_results['test-mae-mean'].min()
    boost_rounds = cv_results['test-mae-mean'].argmin()
    print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
    if mean_mae < min_mae:
        min_mae = mean_mae
        best_params = (max_depth, subsample)

    print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))
    
####################################################################################################################    
gridsearch_params = [
    (alpha,gamma)
    for alpha in [1/n for n in range(1,3)]
	for gamma in [1/n for n in range(1,3)]
]*+

params = {}

for alpha, gamma in gridsearch_params:
  print("CV with alpha={}, gamma={}".format(alpha, gamma))
  params['max_depth'] = 12
  params['subsample'] = 0.8
  params['alpha'] = alpha
  params['gamma'] = gamma
  
  cv_results = xgb.cv(
        params,
        xgtrain,
        1000,
        seed=42,
        nfold=3,
        metrics={'mae'},
        early_stopping_rounds=10
    )
  
  mean_mae = cv_results['test-mae-mean'].min()
  boost_rounds = cv_results['test-mae-mean'].argmin()
  print("\tMAE {} for {} rounds".format(mean_mae, boost_rounds))
  if mean_mae < min_mae:
    min_mae = mean_mae
    best_params = (alpha, gamma)
    
  print("Best params: {}, {}, MAE: {}".format(best_params[0], best_params[1], min_mae))
###########################################################################################################

gridsearch_params = [
    (colsample_bytree,eta)
    for colsample_bytree in [1/n for n in range(1,3)]
	  for eta in [1/(10**n) for n in range(1,4)]
]

params = {}

for colsample_bytree, eta in gridsearch_params:
  print("CV with colsample_bytree={}, eta={}".format(colsample_bytree, eta))
  params['max_depth'] = 12
  params['subsample'] = 0.8
  params['alpha'] = 1

In [ ]:
"""
Reading Train and Test data
"""
# Reading training data
Train_Data = pd.read_csv('train.csv')
## Reading test data
Test_Data = pd.read_csv('test.csv')
# saving ID for submission file.
Submission_Id = Test_Data['id']

r,c = Train_Data.shape   # r -> rows and c  --> columns # 188318 and 130

#Reading y value.
y = np.log(Train_Data['loss'] + 200) 

# Removing ID and LOSS columns from Traina nd ID from Test
Train_Data.drop(['id','loss'], axis=1, inplace=True)
Test_Data.drop(['id'], axis=1, inplace=True)

## Concatinating Train and Test
Train_Test = pd.concat((Train_Data, Test_Data)).reset_index(drop=True)

## Custom metrice : Error calculation
def evalerror(preds, dtrain):
    labels = dtrain.get_label()
    return 'mae', mean_absolute_error(np.exp(preds), np.exp(labels))

## main funtion
if __name__ == '__main__':
  # Filtering categorical dfeatures.
  cat_feature = [n for n in Train_Data.columns if n.startswith('cat')]
  ## Looping through categorical features. 
  for column in cat_feature:
    # For every column , validating the Train classes and test classes equal or not. 
    if Train_Data[column].nunique() != Test_Data[column].nunique():
      ## Reading only unique train class from eac column 
      Unique_classes_Train = set(Train_Data[column].unique())
      ## Reading only unique test class from eac column 
      Unique_classes_Test = set(Test_Data[column].unique())
      ## Classes available in train but not in test 
      missing_train = Unique_classes_Train.difference(Unique_classes_Test)           # set_A.difference(set_B) for (A - B)
      ## Classes available in test but not in train 
      missing_test =  Unique_classes_Test.difference(Unique_classes_Train)
      ## Reading all missing classes from train and test
      All_misisng = missing_train.union(missing_test)


      # Replace all misisng categories with a common category instead of removing. 
      def missing_common(x):
        if x in All_misisng:
          return np.nan
        return x
      
      ## missing classes is replaced with null values. 
      ## Pandas factorize replaces all categorical variables by integers, and NaN with -1, and xgboost treats them as the same category.
      Train_Test[column] = Train_Test[column].apply(lambda x: missing_common(x), 1)   # Axis 1 :: columns
    
    ## factorize the categorical features. 
    Train_Test[column] = pd.factorize(Train_Test[column].values, sort=True)[0]
  
  ## Separating the Train and Test data.
  Train_Data_final = Train_Test.iloc[:r, :]
  Test_Data_final = Train_Test.iloc[r:, :]

   ## K-fold cross validation. 
  kf = KFold(n_splits=3)
  prediction = np.zeros(Submission_Id.shape)
  
  ## Train data divided into 3 folds with 2 for training and 1 for testing 
  for train_index, test_index in kf.split(Train_Data_final):
    X_train, X_test = Train_Data_final.iloc[train_index], Train_Data_final.iloc[test_index]
    y_train, y_test = y.iloc[train_index], y.iloc[test_index]

    params = {
            'min_child_weight': 1,             # Default valaue
            'eta': 0.01,                       # Hyper parameter
            'colsample_bytree': 0.5,            # Hyper parameter
            'max_depth': 12,                    # Hyper parameter
            'subsample': 0.8,                   # Hyper parameter
            'alpha': 1,                         # Hyper parameter
            'gamma': 1,                         # Hyper parameter
            'silent': 1,
            'verbose_eval': True,
             'seed': 2020,
        }

     ## Converting DataFrame to Dmatrix, DMatrix is an internal data structure that is used by XGBoost, which is optimized for both memory efficiency and training speed.
    xgtrain = xgb.DMatrix(X_train, label=y_train)
    xgtest  = xgb.DMatrix(X_test, label=y_test)
    xgtest_final = xgb.DMatrix(Test_Data_final)